Classification of the bacteria data using three different classifiers:
1. SVM
2. RandomForest with decision stumps
3. AdaBoost with decision stumps.

The data is each replicate of each species along the rows, and the fluorescense spectra readings at the log growth phase along the columns. 

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/mnt/d/dev/summer-bursary-2018/bacteria'

In [3]:
os.listdir('data')

['16ms_32ms_growth_phase_spectra.csv',
 '16_ms_lag_codes.csv',
 'bacteria.csv',
 'Classific.py',
 'graphs.pdf',
 'wavelengths.csv']

In [4]:
df = pd.read_csv('data/bacteria.csv', header=[0, 1, 2], index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Columns: 135 entries, (lag, bc, 01) to (stat, se, 07)
dtypes: float64(135)
memory usage: 1.1 MB


# Preprocessing

In [5]:
log = df['log'].T
log

0         1         2         3         4     \
species replicate                                                         
bc      01        -1.823470e+00  0.210400  0.631193  0.420815  3.577017   
        02        -4.455700e-01 -1.209390  0.572870  4.392246  1.973376   
        03        -4.751633e-01 -0.135760  1.425496 -0.067887  3.122725   
        04         2.085158e+00  0.536180 -1.489386  3.276830  1.132023   
        05         1.711495e+00  0.305624 -0.122250 -2.628490 -1.222550   
        06         5.918250e-01  0.328791  0.263034 -1.381000 -0.920670   
        07        -8.084700e-01 -0.995030  0.248757  1.243856  2.923092   
        08         5.402800e-01 -0.600320 -0.420220  0.660376 -0.600340   
        09        -3.118100e-01 -0.561250  0.935429  3.180636  2.432262   
        10         3.067020e-01  0.061425  1.594791  2.523547 -4.616560   
        11        -3.437090e+00 -0.528780  0.396589  1.850829  0.132208   
        12         2.601520e-01  0.845490  1.235727  2.341503 -0.325220   
ec      01        -1.086490e+00 -1.278220  0.575195  1.789606  3.195759   
        02        -9.621601e-01  0.128289  0.769724  2.501744 -0.449036   
        03         5.632230e-01 -0.915230  0.633628  0.422439  0.563257   
        04         9.230000e-07 -1.605890  2.964716  0.494150  0.247073   
        05         7.693800e-01  0.192344 -2.308140  1.282365  1.731210   
lm      01        -1.309300e-01  4.709995 -1.243040 -5.647890 -8.933080   
        02         1.258058e+00 -2.118820  0.132426 -0.397290  3.112194   
        03        -1.139220e+00 -0.822760 -0.316450  3.797592  3.291277   
pa      01         3.808260e-01 -0.507760  0.888587  0.571274  2.094664   
        02        -4.529804e-01  1.164800  0.711819  1.100136  0.647133   
        03         9.560000e-07 -0.627948 -0.767493  0.558205  1.674651   
        04         1.529802e+00 -0.446190 -1.147340 -2.486030 -0.509970   
        05        -4.412800e-01 -0.504310  1.197751 -0.441300 -1.765240   
        06         1.156812e+00 -0.121770 -1.217700  0.365323  2.374646   
        07        -1.966600e-01 -0.131110  0.852198 -0.196670 -1.770060   
        08        -6.211000e-02 -0.559000  0.434780  0.496915 -1.055960   
sa      01         1.126333e+00 -0.750880  0.062574  1.188970  3.754666   
        02         9.270535e-01  1.986511  1.059476  0.331113  3.112396   
        03         5.145850e-01 -1.543730 -1.157800  0.964893  2.573070   
        04         2.822809e+00  8.592621 -0.188260 -3.964090 -3.965050   
        05        -2.579490e+00 -0.692060 -0.943720  2.957123  1.824630   
        06         1.045864e+00  0.326841  5.687094  3.081425 -5.968470   
        07        -1.899700e-01 -1.329840  0.823225  2.089861  0.443312   
        08        -1.313400e-01  0.394027 -1.707460  2.627011  4.597338   
se      01         6.468450e-01 -1.617090  0.646836  0.258749  1.552515   
        02        -4.663400e-01 -0.932670 -0.932670 -0.799470 -0.199870   
        03         3.790200e-01 -1.010715 -1.073889 -0.063173  0.379046   
        04        -3.871300e-01  0.129045  0.516185  0.322631 -0.967900   
        05         7.872760e-01  0.242244  0.242238  1.756314 -0.181680   

                        5         6             7         8             9     \
species replicate                                                              
bc      01         -1.823570  1.543025  1.332621e+00  1.332612  2.314516e+00   
        02          3.501094  1.973367  2.291637e+00  0.954865  1.464087e+00   
        03          0.271546  0.067870  3.394230e-01  2.579618  2.104424e+00   
        04          1.727810  1.608614  1.608642e+00 -0.774528  2.442737e+00   
        05          3.178665  1.772723  2.689683e+00  0.305657 -1.833800e-01   
        06         -0.263040  0.789153  4.770000e-06  0.526096  2.630463e+00   
        07          1.243862  1.741436  1.990201e+00  0.435362  6.219290e-01   
        08          1.260745  0.180102  2.701590e+00  1.620941  2.581469e+00   
 

In [6]:
log.isnull().values.any()

False

In [52]:
X = log.values

In [8]:
y = X.reset_index()['species']
y.head()

0    bc
1    bc
2    bc
3    bc
4    bc
Name: species, dtype: object

In [9]:
y.describe()

count     41
unique     6
top       bc
freq      12
Name: species, dtype: object

In [10]:
print(y)

0     bc
1     bc
2     bc
3     bc
4     bc
5     bc
6     bc
7     bc
8     bc
9     bc
10    bc
11    bc
12    ec
13    ec
14    ec
15    ec
16    ec
17    lm
18    lm
19    lm
20    pa
21    pa
22    pa
23    pa
24    pa
25    pa
26    pa
27    pa
28    sa
29    sa
30    sa
31    sa
32    sa
33    sa
34    sa
35    sa
36    se
37    se
38    se
39    se
40    se
Name: species, dtype: object


In [11]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

print(X[:5])
print(y[:5])

                           0         1         2         3         4     \
species replicate                                                         
ec      01        -1.086490e+00 -1.278220  0.575195  1.789606  3.195759   
        04         9.230000e-07 -1.605890  2.964716  0.494150  0.247073   
sa      03         5.145850e-01 -1.543730 -1.157800  0.964893  2.573070   
se      01         6.468450e-01 -1.617090  0.646836  0.258749  1.552515   
        03         3.790200e-01 -1.010715 -1.073889 -0.063173  0.379046   

                       5         6         7         8         9       ...     \
species replicate                                                      ...      
ec      01         1.470073  4.026641  2.364880  0.255662  2.300933    ...      
        04        -0.494150  0.802997  3.211972  2.285468  1.482439    ...      
sa      03        -0.257300  0.643261  2.315750 -0.771920  0.514613    ...      
se      01         2.652206 -0.582190  1.293756 -1.487830 -1.552490  

# PCA + SVM

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(X)

X_pca = pca.transform(X)
print(X_pca.shape)

(41, 1)


In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, cross_val_score, GridSearchCV

param_grid = {
    'kernel': ['linear', 'rbf'],
    'gamma': [10 ** -n for n in range(6)],
    'C': [10 ** n for n in range(-5, 2)]
}

cv = StratifiedKFold(n_splits=3)
clf = SVC()

grid_search = GridSearchCV(clf, param_grid, cv=cv, verbose=10, n_jobs=4)
grid_search.fit(X_pca, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 3 folds for each of 84 candidates, totalling 252 fits
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=1e-05, gamma=0.1, kernel=linear ...............................
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.36363636363636365, total=   0.0s
[CV]  C=1e-05, gamma=1, kernel=linear, score=0.45454545454545453, total=   0.1s
[CV] C=1e-05, gamma=0.1, kernel=linear ...............................
[CV] ....... C=1e-05, gamma=1, kernel=linear, score=0.4, to

[Parallel(n_jobs=4)]: Batch computation too fast (0.0385s.) Setting batch_size=10.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s


[CV]  C=1e-05, gamma=1, kernel=linear, score=0.5333333333333333, total=   0.7s
[CV] C=1e-05, gamma=0.0001, kernel=rbf ...............................
[CV]  C=1e-05, gamma=0.0001, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=1e-05, gamma=0.0001, kernel=rbf ...............................
[CV]  C=1e-05, gamma=0.0001, kernel=rbf, score=0.36363636363636365, total=   0.0s
[CV] C=1e-05, gamma=1e-05, kernel=linear .............................
[CV]  C=1e-05, gamma=0.1, kernel=linear, score=0.5333333333333333, total=   0.7s
[CV] C=0.0001, gamma=1, kernel=linear ................................
[CV] ... C=1e-05, gamma=1e-05, kernel=linear, score=0.4, total=   0.1s
[CV] C=1e-05, gamma=1e-05, kernel=linear .............................
[CV]  C=1e-05, gamma=0.001, kernel=linear, score=0.5333333333333333, total=   0.6s
[CV] C=1e-05, gamma=0.001, kernel=linear .............................
[CV]  C=1e-05, gamma=0.01, kernel=linear, score=0.5333333333333333, total=   0.7s
[CV] C=1e-05, 

[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:    1.7s


[CV] .... C=0.0001, gamma=0.1, kernel=linear, score=0.4, total=   0.8s
[CV] C=0.0001, gamma=0.1, kernel=linear ..............................
[CV] ...... C=0.0001, gamma=1, kernel=linear, score=0.4, total=   0.8s
[CV] C=0.0001, gamma=1, kernel=linear ................................
[CV] . C=0.0001, gamma=0.0001, kernel=linear, score=0.4, total=   0.9s
[CV] C=0.0001, gamma=0.0001, kernel=linear ...........................
[CV]  C=0.0001, gamma=0.01, kernel=linear, score=0.5333333333333333, total=  11.3s
[CV] C=0.0001, gamma=0.01, kernel=linear .............................
[CV]  C=0.0001, gamma=0.1, kernel=linear, score=0.5333333333333333, total=  11.5s
[CV] C=0.0001, gamma=0.1, kernel=linear ..............................
[CV]  C=0.0001, gamma=1, kernel=linear, score=0.5333333333333333, total=  11.4s
[CV] C=0.0001, gamma=1e-05, kernel=linear ............................
[CV]  C=0.0001, gamma=0.01, kernel=linear, score=0.45454545454545453, total=   1.0s
[CV] C=0.0001, gamma=0.01, kerne

...

[CV]  C=10, gamma=0.1, kernel=linear, score=0.4666666666666667, total=  40.6s
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV]  C=10, gamma=1, kernel=linear, score=0.36363636363636365, total= 2.2min
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV]  C=1, gamma=1e-05, kernel=linear, score=0.5333333333333333, total= 5.0min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.36363636363636365, total=   0.0s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV]  C=10, gamma=0.01, kernel=linear, score=0.4666666666666667, total=  40.4s
[CV] C=10, gamma=0.01, kernel=lin

[Parallel(n_jobs=4)]: Done 232 tasks      | elapsed: 30.0min


[CV]  C=10, gamma=0.01, kernel=linear, score=0.36363636363636365, total= 2.2min
[CV] C=10, gamma=0.001, kernel=linear ................................
[CV]  C=10, gamma=0.1, kernel=linear, score=0.5333333333333333, total= 5.7min
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV]  C=10, gamma=0.001, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV]  C=10, gamma=0.001, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV]  C=10, gamma=0.001, kernel=rbf, score=0.36363636363636365, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV]  C=10, gamma=0.0001, kernel=linear, score=0.4666666666666667, total=  35.4s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV]  C=10, gamma=0.001, kernel=linear, score=0.36363636363636365, total= 2.0min
[CV] C=10, gamma=0.0

[Parallel(n_jobs=4)]: Done 252 out of 252 | elapsed: 38.1min finished


0.5121951219512195
{'C': 0.01, 'gamma': 1, 'kernel': 'linear'}


In [14]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.42 (+/- 0.20)
CPU times: user 18min 12s, sys: 0 ns, total: 18min 12s
Wall time: 18min 13s


In [15]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.60 (+/- 0.27)
CPU times: user 688 ms, sys: 15.6 ms, total: 703 ms
Wall time: 680 ms


# Random Forest with Decision Stumps

In [16]:
%%time

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/home/anthony/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Accuracy: 0.49 (+/- 0.09)
CPU times: user 34.9 s, sys: 172 ms, total: 35.1 s
Wall time: 35.1 s


In [17]:
%%time

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.48 (+/- 0.09)
CPU times: user 36.8 s, sys: 281 ms, total: 37.1 s
Wall time: 37.1 s


# AdaBoost

In [18]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.49 (+/- 0.09)
CPU times: user 40.4 s, sys: 15.6 ms, total: 40.4 s
Wall time: 40.4 s


In [19]:
%%time

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.39 (+/- 0.23)
CPU times: user 1min 59s, sys: 0 ns, total: 1min 59s
Wall time: 1min 59s
